##Badging Design Doc

In [22]:
%reload_ext autoreload
%autoreload 2

import svgwrite
import IPython

### Badge Code

note: loading this from the app codebase

In [23]:
# %load '../lib/badge.py'

import svgwrite


class Badge():
    '''
    generate a badge for some text pair

    text object (one for left, one for right) is defined by
    a string (for display), a background color, a text color,
    and a link.

    ex:
        {
            "text": "github",
            "background": "#eee",
            "href": "http://www.github.com"
        }

    supports shields.io plastic, flat round and flat square styles
    and href links for left and right.
    '''

    # default sizing
    _pad = 8
    _sep = 4
    _height = 20

    def __init__(self, left, right, style="flat"):
        self.left = left
        self.right = right
        self.style = style

        if style == 'plastic':
            self._height = 18

    def _text_width(self, text):
        return 7 * len(text)

    def _calculate_widths(self):
        right_width = self._sep + \
            self._text_width(self.right.get("text")) + \
            self._pad
        left_width = self._pad + \
            self._text_width(self.left.get("text")) + \
            self._sep
        return left_width, right_width, left_width + right_width

    # svg fragments
    def _generate_linear_gradient(self):
        lg = svgwrite.gradients.LinearGradient(
            end=(0, "100%"),
            **{"id": "b"}
        )
        if self.style == 'flat round':
            lg.add_stop_color(offset=0, color="#bbb", opacity=".1")
            lg.add_stop_color(offset=1, opacity=".1")
        elif self.style == 'plastic':
            lg.add_stop_color(offset=0, color="#fff", opacity=".7")
            lg.add_stop_color(offset=0.1, color="#aaa", opacity=".1")
            lg.add_stop_color(offset=0.9, opacity=".3")
            lg.add_stop_color(offset=1, opacity=".5")
        return lg

    def _generate_mask(self, tw):
        mask = svgwrite.masking.Mask(**{"id": "a"})
        mask_rect = svgwrite.shapes.Rect(
            size=(tw, self._height),
            rx=4 if self.style == 'plastic' else 3,
            **{"fill": "#fff"}
        )
        mask.add(mask_rect)
        return mask

    def _generate_background(self, lw, rw, tw):
        if self.style == 'flat square':
            group = svgwrite.container.Group(
                **{"shape-rendering": "crispEdges"})
            group.add(svgwrite.shapes.Rect(
                size=(lw, self._height),
                **{"fill": self.left.get('background')})
            )
            group.add(svgwrite.shapes.Rect(
                size=(rw, self._height),
                insert=(lw, 0),
                **{"fill": self.right.get('background')})
            )
            return group

        # and the other two
        group = svgwrite.container.Group(**{"mask": "url(#a)"})
        group.add(svgwrite.shapes.Rect(
            size=(lw, self._height),
            **{"fill": self.left.get('background')})
        )
        group.add(svgwrite.shapes.Rect(
            size=(rw, self._height),
            insert=(lw, 0),
            **{"fill": self.right.get('background')})
        )
        group.add(
            svgwrite.shapes.Rect(
                size=(tw, self._height),
                **{"fill": "url(#b)"}
            )
        )
        return group

    def _generate_text_group(self, lw, rw):
        '''
        NOTE: the y values are taken from the shields.io templates
        '''
        tg = {
            "fill": "#fff",
            "text-anchor": "middle",
            "font-family": "DejaVu Sans,Verdana,Geneva,sans-serif",
            "font-size": "11"
        }
        group = svgwrite.container.Group(**tg)

        if self.style == 'flat square':
            group.add(svgwrite.text.Text(
                text=self.left.get('text'),
                x=[str((lw / 2) + 1)],
                y=['14']
            ))
            group.add(svgwrite.text.Text(
                text=self.right.get('text'),
                x=[str(lw + (rw / 2) - 1)],
                y=['14']
            ))
        elif self.style == 'flat round':
            # left
            lx = (lw / 2)  # + 1
            group.add(svgwrite.text.Text(
                text=self.left.get('text'),
                x=[str(lx)],
                y=['15'],
                **{"fill": "#010101", "fill-opacity": ".3"}
            ))
            group.add(svgwrite.text.Text(
                text=self.left.get('text'),
                x=[str(lx)],
                y=['14']
            ))

            # right
            lx = lw + (rw / 2) - 1
            group.add(svgwrite.text.Text(
                text=self.right.get('text'),
                x=[str(lx)],
                y=['15'],
                **{"fill": "#010101", "fill-opacity": ".3"}
            ))
            group.add(svgwrite.text.Text(
                text=self.right.get('text'),
                x=[str(lx)],
                y=['14']
            ))
        elif self.style == 'plastic':
            # left
            lx = (lw / 2) + 1
            group.add(svgwrite.text.Text(
                text=self.left.get('text'),
                x=[str(lx)],
                y=['14'],
                **{"fill": "#010101", "fill-opacity": ".3"}
            ))
            group.add(svgwrite.text.Text(
                text=self.left.get('text'),
                x=[str(lx)],
                y=['13']
            ))

            # right
            lx = lw + (rw / 2) - 1
            group.add(svgwrite.text.Text(
                text=self.right.get('text'),
                x=[str(lx)],
                y=['14'],
                **{"fill": "#010101", "fill-opacity": ".3"}
            ))
            group.add(svgwrite.text.Text(
                text=self.right.get('text'),
                x=[str(lx)],
                y=['13']
            ))

        return group

    def generate_badge(self):
        # now with styles
        lw, rw, tw = self._calculate_widths()

        # build the svg
        extras = {"xmlns": "http://www.w3.org/2000/svg"}
        if 'href' in self.left or 'href' in self.right:
            extras["xmlns:xlink"] = "http://www.w3.org/1999/xlink"
        svg = svgwrite.container.SVG(
            size=(tw, self._height),
            **extras
        )

        if self.style in ['flat round', 'plastic']:
            svg.add(self._generate_linear_gradient())

            svg.add(self._generate_mask(tw))

        svg.add(self._generate_background(lw, rw, tw))

        svg.add(self._generate_text_group(lw, rw))

        return svg.tostring()


### The Design Options

In [24]:
# the left side, always esip
_left = {
    "text": "ESIP",
    "background": "#3FA1B9",
    "href": "http://www.esipfed.org"
}

In [25]:
# the right side options
right = {
    "text": "Funded",
    "background": "#A1D3E4"
}


In [26]:
badger = Badge(_left, right, style='plastic')
svg = badger.generate_badge()

print svg

IPython.display.SVG(data=svg)

<svg height="18" width="94" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><defs /><linearGradient id="b" x2="0" y2="100%"><stop offset="0" stop-color="#fff" stop-opacity=".7" /><stop offset="0.1" stop-color="#aaa" stop-opacity=".1" /><stop offset="0.9" stop-opacity=".3" /><stop offset="1" stop-opacity=".5" /></linearGradient><mask id="a"><rect fill="#fff" height="18" rx="4" width="94" x="0" y="0" /></mask><g mask="url(#a)"><rect fill="#3FA1B9" height="18" width="40" x="0" y="0" /><rect fill="#A1D3E4" height="18" width="54" x="40" y="0" /><rect fill="url(#b)" height="18" width="94" x="0" y="0" /></g><g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle"><text fill="#010101" fill-opacity=".3" x="21" y="14">ESIP</text><text x="21" y="13">ESIP</text><text fill="#010101" fill-opacity=".3" x="66" y="14">Funded</text><text x="66" y="13">Funded</text></g></svg>


In [25]:
print svg

<svg height="20" width="94" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><defs /><linearGradient id="b" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1" /><stop offset="1" stop-opacity=".1" /></linearGradient><mask id="a"><rect fill="#fff" height="20" rx="3" width="94" x="0" y="0" /></mask><g mask="url(#a)"><rect fill="#3FA1B9" height="20" width="40" x="0" y="0" /><rect fill="#A1D3E4" height="20" width="54" x="40" y="0" /><rect fill="url(#b)" height="20" width="94" x="0" y="0" /></g><g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle"><text fill="#010101" fill-opacity=".3" x="21" y="15">ESIP<a target="_blank" xlink:href="http://www.esipfed.org" /></text><text x="21" y="14">ESIP</text><text fill="#010101" fill-opacity=".3" x="66" y="15">Funded</text><text x="66" y="14">Funded</text></g></svg>


In [6]:
svg = """<svg height="20" width="94" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<defs />
<style>
    <![CDATA[
    #llink:hover { fill:url(#b); stroke:#ccc; }
    #rlink:hover { fill:#4183C4; }
  ]]>
    </style>
<linearGradient id="b" x2="0" y2="100%">
<stop offset="0" stop-color="#bbb" stop-opacity=".1" />
<stop offset="1" stop-opacity=".1" />
</linearGradient>
<mask id="a">
<rect fill="#fff" height="20" rx="3" width="94" x="0" y="0" />
</mask>
<g mask="url(#a)">
<rect fill="#3FA1B9" height="20" width="40" x="0" y="0" />
<rect fill="#A1D3E4" height="20" width="54" x="40" y="0" />
<rect fill="url(#b)" height="20" width="94" x="0" y="0" />
</g>
<g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle">
<text fill="#010101" fill-opacity=".3" x="21" y="15">ESIP</text>
<text x="21" y="14">ESIP</text>
<text fill="#010101" fill-opacity=".3" x="66" y="15">Funded</text>
<text x="66" y="14">Funded</text>
</g>
<a xlink:href="https://github.com/badges/shields/fork">
        <rect id="llink" stroke="#d5d5d5" fill="url(#a)" height="20" width="40" x="0" y="0" rx="2"/>
    </a>
</svg>
"""

IPython.display.SVG(data=svg)

In [3]:
IPython.display.SVG(url="https://img.shields.io/github/forks/badges/shields.svg?style=social&amp;label=Fork")

In [ ]:
<img src="https://img.shields.io/github/forks/badges/shields.svg?style=social&amp;label=Fork" alt="">


# social with links
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="90" height="20">
    <style>
    <![CDATA[
    #llink:hover { fill:url(#b); stroke:#ccc; }
    #rlink:hover { fill:#4183C4; }
  ]]>
    </style>
    <linearGradient id="a" x2="0" y2="100%">
        <stop offset="0" stop-color="#fcfcfc" stop-opacity="0"/>
        <stop offset="1" stop-opacity=".1"/>
    </linearGradient>
    <linearGradient id="b" x2="0" y2="100%">
        <stop offset="0" stop-color="#ccc" stop-opacity=".1"/>
        <stop offset="1" stop-opacity=".1"/>
    </linearGradient>
    <g stroke="#d5d5d5">
        <rect stroke="none" fill="#fcfcfc" x=".5" y=".5" width="56" height="19" rx="2"/>
        <rect y=".5" x="62.5" width="27" height="19" rx="2" fill="#fafafa"/>
        <path stroke="#fafafa" d="M62 7.5h.5v5H62z"/>
        <path d="M62.5 6.5l-3 3v1l3 3" stroke="d5d5d5" fill="#fafafa"/>
    </g>
    <image x="5" y="3" width="14" height="14" xlink:href=""/>
    <g fill="#333" text-anchor="middle" font-family="Helvetica Neue,Helvetica,Arial,sans-serif" font-weight="700" font-size="11">
        <text x="36.5" y="15" fill="#fff">Forks</text>
        <text x="36.5" y="14">Forks</text>
        <text x="75.5" y="15" fill="#fff">288</text>
        <a xlink:href="https://github.com/badges/shields/network">
            <text id="rlink" x="75.5" y="14">288</text>
        </a>
    </g>
    <a xlink:href="https://github.com/badges/shields/fork">
        <rect id="llink" stroke="#d5d5d5" fill="url(#a)" x=".5" y=".5" width="56" height="19" rx="2"/>
    </a>
</svg>

# plastic style
<svg xmlns="http://www.w3.org/2000/svg" width="83" height="18">
    <linearGradient id="b" x2="0" y2="100%">
        <stop offset="0" stop-color="#fff" stop-opacity=".7"/>
        <stop offset=".1" stop-color="#aaa" stop-opacity=".1"/>
        <stop offset=".9" stop-opacity=".3"/>
        <stop offset="1" stop-opacity=".5"/>
    </linearGradient>
    <mask id="a">
        <rect width="83" height="18" rx="4" fill="#fff"/>
    </mask>
    <g mask="url(#a)">
        <path fill="#555" d="M0 0h37v18H0z"/>
        <path fill="#97CA00" d="M37 0h46v18H37z"/>
        <path fill="url(#b)" d="M0 0h83v18H0z"/>
    </g>
    <g fill="#fff" text-anchor="middle" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11">
        <text x="19.5" y="14" fill="#010101" fill-opacity=".3">style</text>
        <text x="19.5" y="13">style</text>
        <text x="59" y="14" fill="#010101" fill-opacity=".3">plastic</text>
        <text x="59" y="13">plastic</text>
    </g>
</svg>

# flat square
<svg xmlns="http://www.w3.org/2000/svg" width="116" height="20">
    <g shape-rendering="crispEdges">
        <path fill="#555" d="M0 0h37v20H0z"/>
        <path fill="#97CA00" d="M37 0h79v20H37z"/>
    </g>
    <g fill="#fff" text-anchor="middle" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11">
        <text x="19.5" y="14">style</text>
        <text x="75.5" y="14">flat-squared</text>
    </g>
</svg>

# flat rounded (default)
<svg xmlns="http://www.w3.org/2000/svg" width="65" height="20">
    <linearGradient id="b" x2="0" y2="100%">
        <stop offset="0" stop-color="#bbb" stop-opacity=".1"/>
        <stop offset="1" stop-opacity=".1"/>
    </linearGradient>
    <mask id="a">
        <rect width="65" height="20" rx="3" fill="#fff"/>
    </mask>
    <g mask="url(#a)">
        <path fill="#555" d="M0 0h37v20H0z"/>
        <path fill="#97CA00" d="M37 0h28v20H37z"/>
        <path fill="url(#b)" d="M0 0h65v20H0z"/>
    </g>
    <g fill="#fff" text-anchor="middle" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11">
        <text x="18.5" y="15" fill="#010101" fill-opacity=".3">style</text>
        <text x="18.5" y="14">style</text>
        <text x="50" y="15" fill="#010101" fill-opacity=".3">flat</text>
        <text x="50" y="14">flat</text>
    </g>
</svg>